In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataval.dataset import WeatherDataset
from dataval.train import Trainer

import os
import matplotlib.pyplot as plt
import pandas as pd

# Explore Corruptions

We are going to introduce some corruptions into our data, with varying degrees of errors. The purpose of this exercise is to learn that corruptions vary in:

* How hard they are to catch
* How much they actually impact downstream accuracy

We will use the continual training pipeline from the first notebook, and the corruption utility functions in `dataval/dataset.py`.

In [3]:
# Load dataset

ds = WeatherDataset(os.path.join(os.getcwd(), "canonical-partitioned-dataset"), sample_frac=0.2)

In [4]:
def train_and_test(train_df, test_df):
    X_train, y_train = ds.split_feature_label(train_df)

    catboost_hparams = {"depth": 5, "iterations": 250, "learning_rate": 0.03, "loss_function": "RMSE"}
    continual_t = Trainer(catboost_hparams)
    continual_t.fit(X_train, y_train, verbose=False)
    print(f"Train MSE for partition {ds.get_partition_key(train_df)}: {continual_t.score(X_train, y_train)}")

    # Evaluate
    X_test, y_test = ds.split_feature_label(test_df)
    print(f"Test MSE for partition {ds.get_partition_key(test_df)}: {continual_t.score(X_test, y_test)}")
    
    return continual_t

## Obvious Corruptions

Data is obviously corrupted if it is:

* Denoted with a missing value (i.e., nan)
* Violating nonnegativity constraints (e.g., negative pressure value)
* Doesn't type check

We will corrupt columns in the same sensor group.

In [5]:
train_df = ds.load(ds.get_partition_keys()[0])
test_df = ds.load(ds.get_partition_keys()[1])

In [6]:
# Establish baseline MSE with clean train and test data

t = train_and_test(train_df, test_df)

print()
print(t.get_feature_importance().head(5))

Train accuracy for partition 2018_35: 4.074896379948685
Test accuracy for partition 2018_36: 4.797844923928596

                              feature  importance
6            cmc_0_0_0_2_interpolated   19.153059
89   gfs_temperature_sea_interpolated   15.313095
87                gfs_temperature_sea   12.796032
109               wrf_t2_interpolated   11.163675
8                         cmc_0_0_0_2    6.928703


### Missing Value Corruption

First, we corrupt just 5% of the test data for the `cmc` sensor group. Note how much worse the test performance is!

In [7]:
corrupted_test_df, _ = ds.corrupt_null(test_df, "cmc", corruption_rate=0.05)

t = train_and_test(train_df, corrupted_test_df)
print()
print(t.get_feature_importance().head(5))

Train accuracy for partition 2018_35: 4.074896379948685
Test accuracy for partition 2018_36: 7.877114694141433

                              feature  importance
6            cmc_0_0_0_2_interpolated   19.153059
89   gfs_temperature_sea_interpolated   15.313095
87                gfs_temperature_sea   12.796032
109               wrf_t2_interpolated   11.163675
8                         cmc_0_0_0_2    6.928703


It gets even worse when increasing the corruption rate to 20%.

In [8]:
corrupted_test_df, _ = ds.corrupt_null(test_df, "cmc", corruption_rate=0.20)

t = train_and_test(train_df, corrupted_test_df)
print()
print(t.get_feature_importance().head(5))

Train accuracy for partition 2018_35: 4.074896379948685
Test accuracy for partition 2018_36: 17.12374905616156

                              feature  importance
6            cmc_0_0_0_2_interpolated   19.153059
89   gfs_temperature_sea_interpolated   15.313095
87                gfs_temperature_sea   12.796032
109               wrf_t2_interpolated   11.163675
8                         cmc_0_0_0_2    6.928703


Instead of corrupting the test data, maybe we corrupt the train data. Note how the important feature values change, and the test performance is still worse than when cleaning on train data!

In [9]:
corrupted_train_df, _ = ds.corrupt_null(train_df, "cmc", corruption_rate=0.2)

t = train_and_test(corrupted_train_df, test_df)
print()
print(t.get_feature_importance().head(5))

Train accuracy for partition 2018_35: 4.461238723471863
Test accuracy for partition 2018_36: 5.086744083392213

                              feature  importance
89   gfs_temperature_sea_interpolated   32.116077
99                             wrf_t2   11.663983
109               wrf_t2_interpolated   11.081401
87                gfs_temperature_sea   10.171075
90           gfs_temperature_sea_next    6.580485


### Violating Nonnegativity

We corrupt 5% of the test data for the `cmc` sensor group. Test performance is similarly bad.

In [10]:
corrupted_test_df, corrupted_cols = ds.corrupt_nonnegative(test_df, "cmc", corruption_rate=0.05)

print(f"Corrupted columns: {corrupted_cols}")
print()

t = train_and_test(train_df, corrupted_test_df)
print()
print(t.get_feature_importance().head(5))

Corrupted columns: ['cmc_0_0_0_1000', 'cmc_0_0_0_2_interpolated', 'cmc_0_0_0_2_next', 'cmc_0_0_0_2', 'cmc_0_0_0_500', 'cmc_0_0_0_700', 'cmc_0_0_0_850', 'cmc_0_0_0_925', 'cmc_0_0_6_2', 'cmc_0_0_7_1000', 'cmc_0_0_7_2', 'cmc_0_0_7_500', 'cmc_0_0_7_700', 'cmc_0_0_7_850', 'cmc_0_0_7_925', 'cmc_0_1_0_0', 'cmc_0_1_11_0', 'cmc_0_1_65_0', 'cmc_0_1_66_0', 'cmc_0_1_67_0', 'cmc_0_1_68_0', 'cmc_0_1_7_0', 'cmc_0_3_0_0', 'cmc_0_3_0_0_next', 'cmc_0_3_1_0', 'cmc_0_3_5_500', 'cmc_0_3_5_700', 'cmc_0_3_5_850', 'cmc_0_3_5_925', 'cmc_0_6_1_0', 'cmc_available', 'cmc_horizon_h', 'cmc_timedelta_s', 'cmc_0_1_65_0_next', 'cmc_0_1_66_0_next', 'cmc_0_1_67_0_next', 'cmc_0_1_68_0_grad', 'cmc_0_1_68_0_next']

Train accuracy for partition 2018_35: 4.074896379948685
Test accuracy for partition 2018_36: 7.562930524734463

                              feature  importance
6            cmc_0_0_0_2_interpolated   19.153059
89   gfs_temperature_sea_interpolated   15.313095
87                gfs_temperature_sea   12.796032
1

### Violating Type Checks

We corrupt 5% of the test data for the `cmc` sensor group. Test performance is not as bad.

In [11]:
corrupted_test_df, corrupted_cols = ds.corrupt_typecheck(test_df, "cmc", corruption_rate=0.05)

print(f"Corrupted columns: {corrupted_cols}")
print()

t = train_and_test(train_df, corrupted_test_df)
print()
print(t.get_feature_importance().head(5))

Corrupted columns: ['cmc_0_6_1_0', 'cmc_available', 'cmc_horizon_h', 'cmc_timedelta_s', 'cmc_0_1_68_0_grad']

Train accuracy for partition 2018_35: 4.074896379948685
Test accuracy for partition 2018_36: 4.798883324595749

                              feature  importance
6            cmc_0_0_0_2_interpolated   19.153059
89   gfs_temperature_sea_interpolated   15.313095
87                gfs_temperature_sea   12.796032
109               wrf_t2_interpolated   11.163675
8                         cmc_0_0_0_2    6.928703


## Subtle Corruptions

This is by no means an exhaustive list, but we will explore what the following corruptions do to the model performance:

* Changing units (e.g., wind speed in m/s to km/s)
* For a row, average values within a sensor group (e.g., make all gfs sensors return the same value)
* Pin a value of a sensor for a fraction of rows (e.g., set climate_pressure to the 5th percentile)

### Changing `gfs_temperature` from Celsius to Fahrenheit

Suppose the units corruption changes the `gfs_temperature` sensor values from Celsius to Fahrenheit for 5% of rows. We can see that MSE gets noticeably worse.

In [12]:
corrupted_test_df, corrupted_cols = ds.corrupt_units(test_df, "gfs_temperature", corruption_rate=0.05)

print(f"Corrupted columns: {corrupted_cols}")
print()

t = train_and_test(train_df, corrupted_test_df)

Corrupted columns: ['gfs_temperature_10000', 'gfs_temperature_15000', 'gfs_temperature_20000', 'gfs_temperature_25000', 'gfs_temperature_30000', 'gfs_temperature_35000', 'gfs_temperature_40000', 'gfs_temperature_45000', 'gfs_temperature_5000', 'gfs_temperature_50000', 'gfs_temperature_55000', 'gfs_temperature_60000', 'gfs_temperature_65000', 'gfs_temperature_7000', 'gfs_temperature_70000', 'gfs_temperature_75000', 'gfs_temperature_80000', 'gfs_temperature_85000', 'gfs_temperature_90000', 'gfs_temperature_92500', 'gfs_temperature_95000', 'gfs_temperature_97500', 'gfs_temperature_sea', 'gfs_temperature_sea_grad', 'gfs_temperature_sea_interpolated', 'gfs_temperature_sea_next']

Train accuracy for partition 2018_35: 4.074896379948685
Test accuracy for partition 2018_36: 6.785072725642688


### Averaging sensor values for some rows

Suppose we average `gfs` sensor values for 5% of rows. MSE also gets noticeably worse.

In [13]:
corrupted_test_df, corrupted_cols = ds.corrupt_average(test_df, "gfs", corruption_rate=0.05)

print(f"Corrupted columns: {corrupted_cols}")
print()

t = train_and_test(train_df, corrupted_test_df)

Corrupted columns: ['gfs_2m_dewpoint', 'gfs_a_vorticity', 'gfs_available', 'gfs_cloudness', 'gfs_clouds_sea', 'gfs_horizon_h', 'gfs_humidity', 'gfs_precipitable_water', 'gfs_precipitations', 'gfs_pressure', 'gfs_r_velocity', 'gfs_soil_temperature', 'gfs_soil_temperature_available', 'gfs_temperature_10000', 'gfs_temperature_15000', 'gfs_temperature_20000', 'gfs_temperature_25000', 'gfs_temperature_30000', 'gfs_temperature_35000', 'gfs_temperature_40000', 'gfs_temperature_45000', 'gfs_temperature_5000', 'gfs_temperature_50000', 'gfs_temperature_55000', 'gfs_temperature_60000', 'gfs_temperature_65000', 'gfs_temperature_7000', 'gfs_temperature_70000', 'gfs_temperature_75000', 'gfs_temperature_80000', 'gfs_temperature_85000', 'gfs_temperature_90000', 'gfs_temperature_92500', 'gfs_temperature_95000', 'gfs_temperature_97500', 'gfs_temperature_sea', 'gfs_temperature_sea_grad', 'gfs_temperature_sea_interpolated', 'gfs_temperature_sea_next', 'gfs_timedelta_s', 'gfs_total_clouds_cover_high', 'gfs

### Pinned Value Corruption

Suppose we pin `gfs` sensor values to 1.00 for 5% of rows. MSE increases still, but not as much as the other corruptions.

In [14]:
corrupted_test_df, corrupted_cols = ds.corrupt_pinned(test_df, "gfs", corruption_rate=0.05, pinned_value=1.00)

print(f"Corrupted columns: {corrupted_cols}")
print()

t = train_and_test(train_df, corrupted_test_df)

Corrupted columns: ['gfs_2m_dewpoint', 'gfs_a_vorticity', 'gfs_available', 'gfs_cloudness', 'gfs_clouds_sea', 'gfs_horizon_h', 'gfs_humidity', 'gfs_precipitable_water', 'gfs_precipitations', 'gfs_pressure', 'gfs_r_velocity', 'gfs_soil_temperature', 'gfs_soil_temperature_available', 'gfs_temperature_10000', 'gfs_temperature_15000', 'gfs_temperature_20000', 'gfs_temperature_25000', 'gfs_temperature_30000', 'gfs_temperature_35000', 'gfs_temperature_40000', 'gfs_temperature_45000', 'gfs_temperature_5000', 'gfs_temperature_50000', 'gfs_temperature_55000', 'gfs_temperature_60000', 'gfs_temperature_65000', 'gfs_temperature_7000', 'gfs_temperature_70000', 'gfs_temperature_75000', 'gfs_temperature_80000', 'gfs_temperature_85000', 'gfs_temperature_90000', 'gfs_temperature_92500', 'gfs_temperature_95000', 'gfs_temperature_97500', 'gfs_temperature_sea', 'gfs_temperature_sea_grad', 'gfs_temperature_sea_interpolated', 'gfs_temperature_sea_next', 'gfs_timedelta_s', 'gfs_total_clouds_cover_high', 'gfs

## Takeaways

How can we prevent against the corruptions demonstrated above? We'll want to run various data validation methods. The challenge is having methods work (1) _without knowledge of_ the specific corruption, since we can't anticipate and enumerate all possible corruptions, and (2) flag all corruptions precisely (i.e., no false positives). We'll explore this later.